In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import ee
import geemap
# 导入相关库
import numpy as np
import matplotlib.pyplot as plt
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [3]:
# 加载研究区域
roi = ee.FeatureCollection('users/311605001111/YangtzeCity/wuhan').geometry()
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,9)

# 加载样本集
samples = ee.FeatureCollection('users/311605001111/sample_wuhan2018')
# Map.addLayer(samples, {}, "WuhanTrainingPoints_2020")
print(samples.size().getInfo())

## 在样本集中，各类样本的个数
# clt_water = samples.filter(ee.Filter.eq('waterclass',1))
# print(clt_water.size().getInfo())
# clt_nowater = samples.filter(ee.Filter.eq('waterclass',0))
# print(clt_nowater.size().getInfo())

2154


In [4]:
## 加载影像

# 去云、云阴影、雪掩膜函数
def maskL8sr(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    return image.updateMask(mask)
# landsat影像可视化参数
visParams = {
    'bands': ['B5', 'B6', 'B4'],
    'min': 0,
    'max': 3000,
    'gamma': 1.4,
}

landsat_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(roi) \
    .filterDate('2018-01-01', '2018-12-31') \
    .map(maskL8sr)
Map.addLayer(landsat_images.mosaic().clip(roi),visParams,'landsat image')

# 计算筛选掉的样本集

In [5]:
# K_mean聚类
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
clusterer = ee.Clusterer.wekaKMeans(2).train(samples,bands)
result = samples.cluster(clusterer)

print(result.first().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [114.2550334958226, 30.706413283674824]}, 'id': '00000000000000000354', 'properties': {'AWEI': -3717.25, 'B2': 908, 'B3': 1163, 'B4': 1315, 'B5': 2545, 'B6': 2203, 'B7': 1643, 'Image_id': 'LC08_123039_20180408', 'NDVI': 0.318652849740933, 'cluster': 0, 'mNDWI': -0.308972073677956, 'ndvi_mndwi': 0.627624923418889, 'waterclass': 0}}


In [6]:
#聚类后的水体
clt_water = result.filter(ee.Filter.eq('cluster',0))
print("聚类后的水体样本个数：",clt_water.size().getInfo())
# Map.addLayer(clt_water,{'color': '7CFC00', 'pointSize': 4,'fillColor': '7CFC00'},"clt_water")
#聚类后的非水体
clt_nowater = result.filter(ee.Filter.eq('cluster',1))
print("聚类后的非水体样本个数：",clt_nowater.size().getInfo())
# Map.addLayer(clt_nowater,{'color': 'f40101ff', 'pointSize': 4,'fillColor': 'ef0b0b56'},"clt_nowater")

聚类后的水体样本个数： 1207
聚类后的非水体样本个数： 947


In [7]:
# cluster与waterclass的对比
def Compare(feature):
    cluster = ee.Number(feature.get('cluster'))
    waterclass = ee.Number(feature.get('waterclass'))
    ft = ee.Algorithms.If(cluster.eq(waterclass),feature.set({'eq':1}),feature.set({'eq':0}))
    return ft

right = result.map(Compare).filter(ee.Filter.eq('eq',0))
print(right.size().getInfo())
# Map.addLayer(right,{'color': 'FF8C00', 'pointSize': 4,'fillColor': 'FF8C00'},"right")

error = result.map(Compare).filter(ee.Filter.eq('eq',1))
print(error.size().getInfo())
# Map.addLayer(error,{'color': 'FF00FF', 'pointSize': 4,'fillColor': 'FF00FF'},"error")

164
1990


In [8]:
# 过滤后的样本
filtered_sample1 = ee.Algorithms.If(right.size().gt(error.size()),right,error)
print("筛选后，剩余的样本数： ",ee.FeatureCollection(filtered_sample1).size().getInfo())

land1 = ee.FeatureCollection(filtered_sample1).filter(ee.Filter.eq('waterclass',0))
print("筛选后样本中，非水体的个数： ",land1.size().getInfo())
water1 = ee.FeatureCollection(filtered_sample1).filter(ee.Filter.eq('waterclass',1))
print("筛选后样本中，水体的个数： ",water1.size().getInfo())

land1_random = land1.randomColumn().filter(ee.Filter.lt('random', 0.01))  # 15 
water1_random = water1.randomColumn().filter(ee.Filter.lt('random', 0.02)) # 14
# print(water1_random.size().getInfo())

筛选后，剩余的样本数：  1990
筛选后样本中，非水体的个数：  1200
筛选后样本中，水体的个数：  790


In [10]:
# 过滤掉的样本
filtered_sample = ee.Algorithms.If(right.size().lt(error.size()),right,error)
print("筛选掉的样本数： ",ee.FeatureCollection(filtered_sample).size().getInfo())

land = ee.FeatureCollection(filtered_sample).filter(ee.Filter.eq('waterclass',0))
print("筛选掉的样本中，非水体的个数： ",land.size().getInfo())
water = ee.FeatureCollection(filtered_sample).filter(ee.Filter.eq('waterclass',1))
print("筛选掉的样本中，水体的个数： ",water.size().getInfo())

land_random = land.randomColumn(seed = 1).filter(ee.Filter.lt('random', 0.05))  # 7
# print(land_random.size().getInfo())

筛选掉的样本数：  164
筛选掉的样本中，非水体的个数：  157
筛选掉的样本中，水体的个数：  7
8


In [ ]:
# 水体 
id_list = ee.List(water.aggregate_array('Image_id')).getInfo()
print(id_list)

id_list = ee.List(water.aggregate_array('Image_id')).getInfo()
vis_point = {'color': 'red', 'pointSize': 100,'fillColor': 'red'}
for i in id_list:
    id = 'LANDSAT/LC08/C01/T1_SR/' + i
    image = maskL8sr(ee.Image(id)).clip(roi)
    point = water.filter(ee.Filter.eq('Image_id',i))
    Map.addLayer(image,visParams,i[-4:])
    Map.addLayer(point,vis_point,i[-6:])
    print(id)

In [14]:
# 陆地 
id_list = ee.List(land_random.aggregate_array('Image_id')).getInfo()
print(id_list)

id_list = ee.List(land_random.aggregate_array('Image_id')).getInfo()
vis_point = {'color': 'red', 'pointSize': 100,'fillColor': 'red'}
for i in id_list:
    id = 'LANDSAT/LC08/C01/T1_SR/' + i
    image = maskL8sr(ee.Image(id)).clip(roi)
    point = land_random.filter(ee.Filter.eq('Image_id',i))
    Map.addLayer(image,visParams,i[-4:])
    Map.addLayer(point,vis_point,i[-6:])
    print(id)

['LC08_123039_20180627', 'LC08_123039_20181017', 'LC08_122039_20180111', 'LC08_122039_20180417', 'LC08_123039_20181102', 'LC08_122039_20181127', 'LC08_122039_20181127', 'LC08_122039_20181127']
LANDSAT/LC08/C01/T1_SR/LC08_123039_20180627
LANDSAT/LC08/C01/T1_SR/LC08_123039_20181017
LANDSAT/LC08/C01/T1_SR/LC08_122039_20180111
LANDSAT/LC08/C01/T1_SR/LC08_122039_20180417
LANDSAT/LC08/C01/T1_SR/LC08_123039_20181102
LANDSAT/LC08/C01/T1_SR/LC08_122039_20181127
LANDSAT/LC08/C01/T1_SR/LC08_122039_20181127
LANDSAT/LC08/C01/T1_SR/LC08_122039_20181127


In [15]:
print(ee.Feature(land_random.toList(8).get(5)).getInfo())

print(ee.Feature(land_random.toList(8).get(6)).getInfo())

print(ee.Feature(land_random.toList(8).get(7)).getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [114.73338356133098, 30.693209885455573]}, 'id': '0000000000000000076b', 'properties': {'AWEI': -5.5, 'B2': 709, 'B3': 851, 'B4': 775, 'B5': 1020, 'B6': 786, 'B7': 532, 'Image_id': 'LC08_122039_20181127', 'NDVI': 0.136490250696379, 'cluster': 1, 'eq': 0, 'mNDWI': 0.039706780696396, 'ndvi_mndwi': 0.096783469999983, 'random': 0.04981373859295424, 'waterclass': 0}}
{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [114.57574987999064, 30.414814829385783]}, 'id': '00000000000000000770', 'properties': {'AWEI': 593.75, 'B2': 598, 'B3': 664, 'B4': 583, 'B5': 635, 'B6': 429, 'B7': 273, 'Image_id': 'LC08_122039_20181127', 'NDVI': 0.042692939244663, 'cluster': 1, 'eq': 0, 'mNDWI': 0.215004574565416, 'ndvi_mndwi': -0.172311635320753, 'random': 0.04505962637263761, 'waterclass': 0}}
{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [114.54757282043393, 30.42412543238538]}, 'id': '00000000000000000775', '